In [90]:
import requests
import pandas as pd
import json

def get_weather(start_dates,end_dates,code):
  '''
  Returns a csv file with weather data for a specified city code and a given list of start and end dates

        Parameters:
            start_dates (list) : A list of start dates one month apart (API FUNCTIONALITY)
            end_dates   (list) : A list of end dates one month apart (API FUNCTIONALITY)
            code        (str)  : Airport city

        Returns: 
            A saved csv file with weather data

        Example:
          start_dates = ['2018-03-01','2018-04-02','2018-05-02','2018-06-02','2018-07-02']  # N0TE: OFFSET BY 1 DAY EACH START DATE AS QUERY IS INCLUSIVE
          end_dates = ['2018-04-01','2018-05-01','2018-06-01','2018-07-01','2018-08-01']
          code = 'ATL'
          get_weather(start_dates,end_dates,code) # For the period ranging between 2018-03-01 & 2018-08-01

  '''
  df_final = pd.DataFrame(columns=['date','maxtempF','mintempC','mintempF','avgtempC','avgtempF','totalSnow_cm','sunHour','uvIndex','hourly','airport_code'])

  for i , j in zip(end_dates,start_dates):

    url = f"http://api.worldweatheronline.com/premium/v1/past-weather.ashx?q= 41.9803, 87.9090&date={j}&enddate={i}&tp=24&key=b04c3773879a49d3a5f170136212706&format=json"

    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)

    data = json.loads(response.text)

    df = pd.json_normalize(data['data']['weather'])


    df_final = df_final.append(df)


  
  df_final['airport_code'] = code



  df_final.to_csv(f'{code}_final.csv',index=False)
 

In [169]:
start_dates = ['2018-03-01','2018-04-02','2018-05-02','2018-06-02','2018-07-02']  # N0TE: OFFSET BY 1 DAY EACH START DATE AS QUERY IS INCLUSIVE
end_dates = ['2018-04-01','2018-05-01','2018-06-01','2018-07-01','2018-08-01']
get_weather(start_dates,end_dates,'ATL')

In [187]:
#Load returned CSVs from the get_weather function
dat = pd.read_csv('ATL_final.csv')
dat.head(3)

,date,maxtempF,mintempC,mintempF,avgtempC,avgtempF,totalSnow_cm,sunHour,uvIndex,hourly,airport_code,astronomy,maxtempC
0,2018-03-01,52,4,39,8,47,0.0,8.7,3,"[{'time': '24', 'tempC': '11', 'tempF': '52', ...",ATL,"[{'sunrise': '06:44 AM', 'sunset': '05:58 PM',...",11
1,2018-03-02,54,3,38,8,47,0.0,8.7,3,"[{'time': '24', 'tempC': '12', 'tempF': '54', ...",ATL,"[{'sunrise': '06:42 AM', 'sunset': '05:59 PM',...",12
2,2018-03-03,45,4,39,6,42,0.0,11.6,3,"[{'time': '24', 'tempC': '7', 'tempF': '45', '...",ATL,"[{'sunrise': '06:41 AM', 'sunset': '06:00 PM',...",7


In [188]:
#Check what each hourly value looks like
dat['hourly'][1]

"[{'time': '24', 'tempC': '12', 'tempF': '54', 'windspeedMiles': '11', 'windspeedKmph': '18', 'winddirDegree': '282', 'winddir16Point': 'WNW', 'weatherCode': '116', 'weatherIconUrl': [{'value': 'http://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0002_sunny_intervals.png'}], 'weatherDesc': [{'value': 'Partly cloudy'}], 'precipMM': '0.0', 'precipInches': '0.0', 'humidity': '34', 'visibility': '10', 'visibilityMiles': '6', 'pressure': '1007', 'pressureInches': '30', 'cloudcover': '9', 'HeatIndexC': '7', 'HeatIndexF': '45', 'DewPointC': '-8', 'DewPointF': '18', 'WindChillC': '5', 'WindChillF': '42', 'WindGustMiles': '16', 'WindGustKmph': '26', 'FeelsLikeC': '5', 'FeelsLikeF': '42', 'uvIndex': '3'}]"

In [189]:
# Function to remove string literal that is enclosing our hourly column
import ast

def parse_hourly(row):
  return ast.literal_eval(row)

In [190]:
#Remove the string literal that is enclosing our hourly column
dat['hourly'] = dat.apply(lambda row: parse_hourly(row['hourly']), axis=1)

In [191]:
#Check what each hourly value looks like now (no string)
dat['hourly'][1]

[{'DewPointC': '-8',
  'DewPointF': '18',
  'FeelsLikeC': '5',
  'FeelsLikeF': '42',
  'HeatIndexC': '7',
  'HeatIndexF': '45',
  'WindChillC': '5',
  'WindChillF': '42',
  'WindGustKmph': '26',
  'WindGustMiles': '16',
  'cloudcover': '9',
  'humidity': '34',
  'precipInches': '0.0',
  'precipMM': '0.0',
  'pressure': '1007',
  'pressureInches': '30',
  'tempC': '12',
  'tempF': '54',
  'time': '24',
  'uvIndex': '3',
  'visibility': '10',
  'visibilityMiles': '6',
  'weatherCode': '116',
  'weatherDesc': [{'value': 'Partly cloudy'}],
  'weatherIconUrl': [{'value': 'http://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0002_sunny_intervals.png'}],
  'winddir16Point': 'WNW',
  'winddirDegree': '282',
  'windspeedKmph': '18',
  'windspeedMiles': '11'}]

In [192]:
# Explode hourly column dictionary into its own dataframe where dictionary keys are columns
dat_hourly = dat.hourly.apply(pd.Series)

In [193]:
dat_hourly.head(3)

,0
0,"{'time': '24', 'tempC': '11', 'tempF': '52', '..."
1,"{'time': '24', 'tempC': '12', 'tempF': '54', '..."
2,"{'time': '24', 'tempC': '7', 'tempF': '45', 'w..."


In [194]:
#Normalise weatherDesc dictionary into seperate dataframe
dat_hourly = pd.json_normalize(dat_hourly[0])

In [195]:
dat_hourly.head(3)

,time,tempC,tempF,windspeedMiles,windspeedKmph,winddirDegree,winddir16Point,weatherCode,weatherIconUrl,weatherDesc,precipMM,precipInches,humidity,visibility,visibilityMiles,pressure,pressureInches,cloudcover,HeatIndexC,HeatIndexF,DewPointC,DewPointF,WindChillC,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF,uvIndex
0,24,11,52,2,3,211,SSW,113,[{'value': 'http://cdn.worldweatheronline.com/...,[{'value': 'Sunny'}],0.0,0.0,35,10,6,1013,30,5,7,45,-7,19,7,45,3,5,7,45,3
1,24,12,54,11,18,282,WNW,116,[{'value': 'http://cdn.worldweatheronline.com/...,[{'value': 'Partly cloudy'}],0.0,0.0,34,10,6,1007,30,9,7,45,-8,18,5,42,16,26,5,42,3
2,24,7,45,17,27,323,NW,116,[{'value': 'http://cdn.worldweatheronline.com/...,[{'value': 'Partly cloudy'}],0.0,0.0,34,10,6,1019,31,25,5,41,-9,15,1,33,24,38,1,33,3


In [196]:
#Get rid of list bracket format in weatherDesc column
testdf = dat_hourly.explode('weatherDesc')

In [197]:
testdf.head(3)

,time,tempC,tempF,windspeedMiles,windspeedKmph,winddirDegree,winddir16Point,weatherCode,weatherIconUrl,weatherDesc,precipMM,precipInches,humidity,visibility,visibilityMiles,pressure,pressureInches,cloudcover,HeatIndexC,HeatIndexF,DewPointC,DewPointF,WindChillC,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF,uvIndex
0,24,11,52,2,3,211,SSW,113,[{'value': 'http://cdn.worldweatheronline.com/...,{'value': 'Sunny'},0.0,0.0,35,10,6,1013,30,5,7,45,-7,19,7,45,3,5,7,45,3
1,24,12,54,11,18,282,WNW,116,[{'value': 'http://cdn.worldweatheronline.com/...,{'value': 'Partly cloudy'},0.0,0.0,34,10,6,1007,30,9,7,45,-8,18,5,42,16,26,5,42,3
2,24,7,45,17,27,323,NW,116,[{'value': 'http://cdn.worldweatheronline.com/...,{'value': 'Partly cloudy'},0.0,0.0,34,10,6,1019,31,25,5,41,-9,15,1,33,24,38,1,33,3


In [198]:
#Normalise weatherDesc dictionary into seperate dataframe
weatherDescription_df = pd.json_normalize(testdf['weatherDesc'])

In [199]:
weatherDescription_df.head(3)

,value
0,Sunny
1,Partly cloudy
2,Partly cloudy


In [200]:
#Merge weatherDescription_df with dat_hourly on index
dat_hourly_merged = dat_hourly.merge(weatherDescription_df, how = 'outer', left_index =True, right_index=True )

In [201]:
dat_hourly_merged.head(3)

,time,tempC,tempF,windspeedMiles,windspeedKmph,winddirDegree,winddir16Point,weatherCode,weatherIconUrl,weatherDesc,precipMM,precipInches,humidity,visibility,visibilityMiles,pressure,pressureInches,cloudcover,HeatIndexC,HeatIndexF,DewPointC,DewPointF,WindChillC,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF,uvIndex,value
0,24,11,52,2,3,211,SSW,113,[{'value': 'http://cdn.worldweatheronline.com/...,[{'value': 'Sunny'}],0.0,0.0,35,10,6,1013,30,5,7,45,-7,19,7,45,3,5,7,45,3,Sunny
1,24,12,54,11,18,282,WNW,116,[{'value': 'http://cdn.worldweatheronline.com/...,[{'value': 'Partly cloudy'}],0.0,0.0,34,10,6,1007,30,9,7,45,-8,18,5,42,16,26,5,42,3,Partly cloudy
2,24,7,45,17,27,323,NW,116,[{'value': 'http://cdn.worldweatheronline.com/...,[{'value': 'Partly cloudy'}],0.0,0.0,34,10,6,1019,31,25,5,41,-9,15,1,33,24,38,1,33,3,Partly cloudy


In [202]:
#Drop weatherDesc column and any other useless columns
dat_hourly_merged = dat_hourly_merged.drop(columns=['weatherIconUrl','weatherDesc'])
dat_hourly_merged.head(3)

,time,tempC,tempF,windspeedMiles,windspeedKmph,winddirDegree,winddir16Point,weatherCode,precipMM,precipInches,humidity,visibility,visibilityMiles,pressure,pressureInches,cloudcover,HeatIndexC,HeatIndexF,DewPointC,DewPointF,WindChillC,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF,uvIndex,value
0,24,11,52,2,3,211,SSW,113,0.0,0.0,35,10,6,1013,30,5,7,45,-7,19,7,45,3,5,7,45,3,Sunny
1,24,12,54,11,18,282,WNW,116,0.0,0.0,34,10,6,1007,30,9,7,45,-8,18,5,42,16,26,5,42,3,Partly cloudy
2,24,7,45,17,27,323,NW,116,0.0,0.0,34,10,6,1019,31,25,5,41,-9,15,1,33,24,38,1,33,3,Partly cloudy


In [204]:
#Merge dat_hourly with original dat dataframe read from csv
dat_new = dat.merge(dat_hourly_merged, how ='outer', left_index=True, right_index=True)

In [207]:
dat_new.head(3)

,date,maxtempF,mintempC,mintempF,avgtempC,avgtempF,totalSnow_cm,sunHour,uvIndex_x,hourly,airport_code,astronomy,maxtempC,time,tempC,tempF,windspeedMiles,windspeedKmph,winddirDegree,winddir16Point,weatherCode,precipMM,precipInches,humidity,visibility,visibilityMiles,pressure,pressureInches,cloudcover,HeatIndexC,HeatIndexF,DewPointC,DewPointF,WindChillC,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF,uvIndex_y,value
0,2018-03-01,52,4,39,8,47,0.0,8.7,3,"[{'time': '24', 'tempC': '11', 'tempF': '52', ...",ATL,"[{'sunrise': '06:44 AM', 'sunset': '05:58 PM',...",11,24,11,52,2,3,211,SSW,113,0.0,0.0,35,10,6,1013,30,5,7,45,-7,19,7,45,3,5,7,45,3,Sunny
1,2018-03-02,54,3,38,8,47,0.0,8.7,3,"[{'time': '24', 'tempC': '12', 'tempF': '54', ...",ATL,"[{'sunrise': '06:42 AM', 'sunset': '05:59 PM',...",12,24,12,54,11,18,282,WNW,116,0.0,0.0,34,10,6,1007,30,9,7,45,-8,18,5,42,16,26,5,42,3,Partly cloudy
2,2018-03-03,45,4,39,6,42,0.0,11.6,3,"[{'time': '24', 'tempC': '7', 'tempF': '45', '...",ATL,"[{'sunrise': '06:41 AM', 'sunset': '06:00 PM',...",7,24,7,45,17,27,323,NW,116,0.0,0.0,34,10,6,1019,31,25,5,41,-9,15,1,33,24,38,1,33,3,Partly cloudy


In [208]:
#Drop useless columns in dat_new
dat_new.drop(columns=['hourly','astronomy'])

,date,maxtempF,mintempC,mintempF,avgtempC,avgtempF,totalSnow_cm,sunHour,uvIndex_x,airport_code,maxtempC,time,tempC,tempF,windspeedMiles,windspeedKmph,winddirDegree,winddir16Point,weatherCode,precipMM,precipInches,humidity,visibility,visibilityMiles,pressure,pressureInches,cloudcover,HeatIndexC,HeatIndexF,DewPointC,DewPointF,WindChillC,WindChillF,WindGustMiles,WindGustKmph,FeelsLikeC,FeelsLikeF,uvIndex_y,value
0,2018-03-01,52,4,39,8,47,0.0,8.7,3,ATL,11,24,11,52,2,3,211,SSW,113,0.0,0.0,35,10,6,1013,30,5,7,45,-7,19,7,45,3,5,7,45,3,Sunny
1,2018-03-02,54,3,38,8,47,0.0,8.7,3,ATL,12,24,12,54,11,18,282,WNW,116,0.0,0.0,34,10,6,1007,30,9,7,45,-8,18,5,42,16,26,5,42,3,Partly cloudy
2,2018-03-03,45,4,39,6,42,0.0,11.6,3,ATL,7,24,7,45,17,27,323,NW,116,0.0,0.0,34,10,6,1019,31,25,5,41,-9,15,1,33,24,38,1,33,3,Partly cloudy
3,2018-03-04,42,-1,31,3,38,0.0,11.6,1,ATL,6,24,6,42,3,5,185,S,119,0.0,0.0,31,10,6,1022,31,23,3,37,-13,9,2,35,4,7,2,35,1,Cloudy
4,2018-03-05,48,1,33,5,41,0.0,11.6,3,ATL,9,24,9,48,4,6,192,SSW,113,0.0,0.0,33,10,6,1020,31,13,4,40,-11,13,3,38,6,10,3,38,3,Sunny
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2018-07-28,88,23,73,28,82,0.0,14.5,7,ATL,31,24,31,88,6,10,275,W,113,0.0,0.0,20,10,6,1007,30,5,26,78,2,36,27,80,9,14,26,78,7,Sunny
150,2018-07-29,92,25,77,30,86,0.0,14.5,8,ATL,33,24,33,92,6,10,282,WNW,113,0.0,0.0,19,10,6,1007,30,10,27,81,3,38,29,84,9,14,27,81,8,Sunny
151,2018-07-30,93,25,78,31,87,0.0,14.5,8,ATL,34,24,34,93,9,15,272,W,113,0.0,0.0,20,10,6,1008,30,17,28,82,4,39,30,85,12,20,28,82,8,Sunny
152,2018-07-31,90,26,78,29,84,0.0,14.3,7,ATL,32,24,32,90,7,12,231,SW,113,0.0,0.0,25,10,6,1006,30,22,27,80,6,43,28,83,9,15,27,80,7,Sunny
